# Performance Testing for StackRox

This document is an idea of executable documentation. It can be used with VS Code and `Jupyter` plugin. It's pretty handy because showing `Outline` allows faster document navigation.

### Pre-requirement

There are few unconventional tools used in this notebook

1. `rg` - ripgrep is replacement for `grep`. Installation instructions can be found here: https://github.com/BurntSushi/ripgrep#installation
2. `yq` - is command line YAML processor. Installation instructions can be found here: https://github.com/mikefarah/yq#install

### Quick start

Install `k6` tool. To install it, please check the instructions on the project page: https://k6.io/docs/get-started/installation. Below is an example for MacOS.

In [ ]:
brew install k6

Install required NodeJS development libraries.

In [ ]:
npm install

**Run test directly**

In [2]:
export ROX_ADMIN_PASSWORD=$(cat ../../deploy/k8s/central-deploy/password)
export HOST="https://localhost:8000"

In [ ]:
npm run test -- --quiet --out csv=k6-test-result.csv

After that you can check `k6-test-result.csv` for results.

**Run test in docker or kubernetes**

Build a docker image with the following command:

In [ ]:
docker build . -t stackrox/performance-test-runner:latest

Create a namespace and deploy `performance-test-runner`. The next command uses `yq` to set a password in the YAML file. To install it, please check the instructions on the project page: https://github.com/mikefarah/yq

In [ ]:
export KUBECONFIG=./artifacts/kubeconfig

export ROX_ADMIN_PASSWORD=$(cat ../../deploy/k8s/central-deploy/password)

In [ ]:
kubectl create namespace performance-test

In [ ]:
yq e '.spec.template.spec.containers[0].env[0].value = "'"${ROX_ADMIN_PASSWORD}"'"' ./deploy/test-runner-deployment.yaml | kubectl apply -f -

To check if everything is working, you can run the following command.

In [ ]:
kubectl get pods --namespace performance-test

### Collecting results

Get logs because results will be output to stdout inside the container.

In [1]:
export KUBECONFIG=./artifacts/kubeconfig

In [ ]:
kubectl logs --namespace performance-test test-runner -c test-runner > ./test-runner-result_2022-11-07_19-12.log

Extract the results of testing in a CSV file. The following line uses the `rg` command. To install it, please check the instructions on the project page: https://github.com/BurntSushi/ripgrep

In [ ]:
rg 'sac_user=|extra_tags' ./test-runner-result_2022-11-07_19-12.log > ./test-runner-result_2022-11-07_19-12.log.csv

After that, this file can be imported into some spreadsheet for further processing of the results.

1. Simplest option is to create Google Sheets.
2. Copy the content of the `.csv` file and paste into the sheet. You will get a pop-up to split a column into multiple columns. By selecting that, multiple columns will be populated with test result data.
3. From the menu, select `Insert -> Pivot table`. A dialog will automatically select the whole sheet range which we want. Select to create a pivot table on a new sheet.
4. On the new sheet in the pivot table options, select the following:
   - For rows: `url` and `extra_tags`. Disable `Show totals`.
   - For columns: `metric_name`. Disable `Show totals`.
   - For values: `metric_value`. You can select `MEDIAN` or `AVERAGE` function.
   - For the filter, you can disable everything and enable only `http_req_waiting`. That metric represents how long a request waits for a server response. It does not include sending and receiving times.
5. Have fun!

### How to create a new test group

We first need HAR export from requests generated with our browser to create a new test group. The best is to use Firefox because Firefox will export only filtered requests in the `networking` tab. After that, you can execute a conversion from the HAR file into a JavaScript file.

In [ ]:
npm run har-to-k6 -- ~/Downloads/new-test-group.har --output ./groups/awesomeNewPage.js

After that, some manual steps should be done:
1. Remove comments above `import`.
2. Remove the `options` constant.
3. Replace function declaration to accept 3 arguments. i.e. `export function awesomeNewPage(host, headers, tags)`.
4. Remove the `response` variable. Declaration and usage of it.
5. Replace all objects with the `headers` property with the following object `{ headers, tags }`.
6. Change URLs in to use the `host` variable. i.e. `'https://localhost:8001/api/graphql?opname=cvesCount'` should become `` `${host}/api/graphql?opname=cvesCount` ``.
7. Remove `sleep` at the end of the function and also its `import`.

The next step is to include the group in a test. i.e. to add a group into the `tests/testSacScopes.js` test, we should do the following.
1. Import the group into a test file. i.e. `import { awesomeNewPage } from '../groups/awesomeNewPage.js';`
2. Add function call in `runAllGroups`. i.e. `awesomeNewPage(__ENV.HOST, header, tags);`

With that new group is created and added to the test. And after that, we can build a docker image and run it. You can take a look at **Quick start** for instructions.

### How to fetch logs from a GKE cluster

Set your `cluster_name` and correct `timestamp`(s) in the logs query below.

In [ ]:
# Query - time is in UTC
read -r -d '' GC_LOGS_QUERY <<- EOQ
    resource.type="k8s_container"
    labels.k8s-pod/app="test-runner"
    resource.labels.project_id="srox-temp-dev-test"
    resource.labels.location="us-central1-a"
    resource.labels.cluster_name="mt-0711"
    resource.labels.namespace_name="performance-test"
    resource.labels.container_name="test-runner"
    timestamp > "2022-11-07T20:00:00Z"
    timestamp < "2022-11-07T21:00:00Z"
    severity>=DEFAULT
EOQ

gcloud logging read "${GC_LOGS_QUERY}" --format='csv(textPayload)' --order='asc' > test-runner-result_2022-11-07_21-00.log

This file will contain quotes for every line and also extra new lines. That should be cleaned up.

The next command is for MacOS `sed` command. It will remove new lines and quotes around each row.

In [ ]:
cat test-runner-result_2022-11-07_21-00.log | sed -e ':a' -e 'N' -e '$!ba' -e 's/"\n"//g' | sed 's/"//g' | rg 'sac_user=|extra_tags' > test-runner-result_2022-11-07_21-00.log.csv

### Development

To check linting and formatting you can run the following commands:

In [ ]:
npm run format:check
npm run lint:check

And to automatically fix them, you can run the following commands:

In [ ]:
npm run format:fix
npm run lint:fix